In [1]:
from dataset import PointCloud, MultiplePointClouds
import meshplot as mp
import open3d as o3d
import numpy as np
from plyfile import PlyData

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getNumpy( p : PointCloud ):
    coords, normals, sdfs = p.__getitem__(0)

    return coords.cpu().detach().numpy(), normals.cpu().detach().numpy(), sdfs.cpu().detach().numpy()

In [4]:
point_cloud = PointCloud(
    "../data/segmented/test2/normalized/test1_0_curv.ply", batch_size=10, use_curvature=True,
    curvature_fractions=(0.75, 0.20, 0.05), curvature_percentiles=(70, 95)
)

mesh = o3d.io.read_triangle_mesh( "../data/segmented/test2/normalized/test1_0_curv.ply" )

Loading mesh "../data/segmented/test2/normalized/test1_0_curv.ply".
Using curvatures?  YES
Fetching 5 on-surface points per iteration.
Creating point-cloud and acceleration structures.
Done preparing the dataset.


In [ ]:
p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array([0.7,0,0]), return_plot=True )

In [ ]:
points, normals, sdf = getNumpy( point_cloud )

p.add_points( points, c=sdf, shading={'point_size':0.3, 'normalization':True} )

In [ ]:
def _read_ply_with_curvatures(path: str):
    """Reads a PLY file with position, normal and curvature info.

    Note that we expect the input ply to contain x,y,z vertex data, as well
    as nx,ny,nz normal data and the curvature stored in the `quality` vertex
    property.

    Parameters
    ----------
    path: str, PathLike
        Path to the ply file. We except the file to be in binary format.

    Returns
    -------
    mesh: o3d.t.geometry.TriangleMesh
        The fully constructed Open3D Triangle Mesh. By default, the mesh is
        allocated on the CPU:0 device.

    vertices: numpy.array
        The same vertex information as stored in `mesh` returned for
        convenience only.

    See Also
    --------
    PlyData.read, o3d.t.geometry.TriangleMesh
    """

    curvaturas = None
    # Reading the PLY file with curvature info
    with open(path, "rb") as f:
        plydata = PlyData.read(f)
        num_verts = plydata["vertex"].count
        vertices = np.zeros(shape=(num_verts, 7), dtype=np.float32)
        vertices[:, 0] = plydata["vertex"].data["x"]
        vertices[:, 1] = plydata["vertex"].data["y"]
        vertices[:, 2] = plydata["vertex"].data["z"]
        vertices[:, 3] = plydata["vertex"].data["nx"]
        vertices[:, 4] = plydata["vertex"].data["ny"]
        vertices[:, 5] = plydata["vertex"].data["nz"]
        vertices[:, 6] = plydata["vertex"].data["quality"]

        curvaturas = plydata["vertex"].data["quality"]
    
    return curvaturas

curvaturas = _read_ply_with_curvatures("../data/segmented/test2/normalized/test1_0_curv.ply")

import matplotlib.pyplot as plt

plt.figure(0)
plt.hist(curvaturas, bins=50)
plt.show()

In [5]:
point_cloud2 = MultiplePointClouds(
    mesh_paths=[ "../data/segmented/test2/normalized/test1_0_curv.ply" ],
    selector_path= "../data/segmented/test1/normalized/selector.csv",
    batch_size=10,
    use_curvature=True, curvature_fractions=(0.75, 0.2, 0.05), curvature_percentiles=(70, 95)
)

p2 = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array([0.7,0,0]), return_plot=True )

Loading mesh "../data/segmented/test2/normalized/test1_0_curv.ply".
Using curvatures?  YES
Fetching 5 on-surface points per iteration.
Creating point-cloud and acceleration structures.
Done preparing the dataset.


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0519458…

In [32]:
def getNumpy2( p : PointCloud ):
    coords, props = p.__getitem__(0)

    coords = coords['coords'].cpu().detach().numpy()
    normals = props['normals'].cpu().detach().numpy()
    sdfs = props['sdf'].cpu().detach().numpy()
    return coords, normals, sdfs

points, normals, sdf = getNumpy2( point_cloud2 )

p2.add_points( points, c=sdf, shading={'point_size':0.3, 'normalization':True} )

26